In [73]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

In [92]:
def get_stock(stock_name):
  try:
    df = yf.download(stock_name,period="1mo")
    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df.drop(['Adj Close','Volume'], axis=1, inplace= True)
    return df
  except:
    return "stock not found"
df=get_stock('V')

[*********************100%***********************]  1 of 1 completed


In [93]:
df.head()

,Date,Open,High,Low,Close
0,2024-08-21,268.559998,269.269989,267.390015,268.200012
1,2024-08-22,268.940002,269.500000,266.570007,267.940002
2,2024-08-23,268.700012,269.209991,265.309998,267.440002
3,2024-08-26,267.720001,270.369995,267.670013,268.209991
4,2024-08-27,268.519989,270.950012,267.890015,270.720001


In [94]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0

    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.Low[l]>df1.Low[i]):
            pividlow=0
        if(df1.High[l]<df1.High[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

df['pivot'] = df.apply(lambda x: pivotid(df, x.name,2,2), axis=1)

In [95]:
def pointpos(x):
  if x['pivot']==1:
    return x['Low']-1e-3
  elif x['pivot']==2:
    return x['High']+1e-3
  else:
    return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
df.tail()

,Date,Open,High,Low,Close,pivot,pointpos
17,2024-09-16,288.049988,290.899994,287.739990,290.480011,0,NaN
18,2024-09-17,290.899994,293.070007,290.079987,291.559998,2,293.071007
19,2024-09-18,291.839996,292.019989,288.130005,288.480011,0,NaN
20,2024-09-19,291.089996,291.480011,282.869995,285.239990,0,NaN
21,2024-09-20,285.500000,286.999908,283.250000,284.769989,0,NaN


In [96]:
dfpl = df[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [97]:

rows=len(df)
rows
backcandles = 0

candleid = rows-1

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

for i in range(backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].Low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].High)
        xxmax = np.append(xxmax, i) # df.iloc[i].name

#slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
#slmax, intercmax = np.polyfit(xxmax, maxim,1)

slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

print(rmin, rmax)

dfpl = df[backcandles:candleid+1]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot")

xxmin = np.append(xxmin, xxmin[-1]+15)
xxmax = np.append(xxmax, xxmax[-1]+15)
#fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
#fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))

fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()
print(df.iloc[candleid])

0.9974662560053958 0.9987049422582795


Date        2024-09-20 00:00:00
Open                      285.5
High                 286.999908
Low                      283.25
Close                284.769989
pivot                         0
pointpos                    NaN
Name: 21, dtype: object
